# AKS Cookbook

## 🧪 AKS with OpenAI

![visual](visual.png)

Use this lab to quickly Deploy an application that uses OpenAI on Azure Kubernetes Service (AKS).  
It's based on the official [AKS documentation](https://learn.microsoft.com/en-us/azure/aks/open-ai-quickstart?tabs=aoai) with a few adaptations to use the [Bicep Kubernetes extension (preview)](https://learn.microsoft.com/en-us/azure/aks/learn/quick-kubernetes-deploy-bicep-kubernetes-extension?tabs=azure-cli).  

▶️ Click on the `Run All` button to execute all the subsequent steps in sequence, or run each step individually by executing the cells one at a time.

### TOC

- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and connected Azure subscription](#1)
- [2️⃣ Create a new Azure Resource Group or reuse an existing one](#2)
- [2️⃣ Creates AKS and OpenAI resources, and deploys the application using Bicep 🦾](#3)
- [4️⃣ Connect to the AKS cluster](#4)
- [5️⃣ Retrieve the ingress IP for the application](#5)
- [6️⃣ Test the application by making a request to the AI service](#6)
- [🗑️ Clean up resources](#clean)



<a id='0'></a>
### 0️⃣ Initialize notebook variables
You can use this notebook with existing resources or to create the necessary resources.  
Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable).

In [ ]:
import os, time, json, requests, utils

create_resources = True # specify True if you want to create new resources, False to use existing ones

if create_resources:
    # create new resources with the following properties
    deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
    resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
    resource_group_location = "eastus2"
    aks_resource_name = "aks-cluster"
    aks_nodepool_count = 1
    aks_nodepool_vm_size = 'standard_d2s_v3'

    openai_resource_prefix = "openai"
    openai_model_name = "gpt-35-turbo"
    openai_model_version = "0613"
    openai_deployment_name = "gpt-35-turbo"

else:
    # or use the following existing resources
    resource_group_name = ""
    aks_resource_name = ""

utils.print_ok('Notebook initiaized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected Azure subscription
The following commands ensure that you have the latest version of the Azure CLI and relevant extensions installed while also verifying that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    subscription_id = output.json_data['id']
    tenant_id = output.json_data['tenantId']
output = utils.run("az provider register --namespace Microsoft.ContainerService --wait", "Microsoft.ContainerService registered in your subscription", "Failed to register Microsoft.ContainerService")
output = utils.run("az provider register --namespace Microsoft.KubernetesConfiguration --wait", "Microsoft.KubernetesConfiguration registered in your subscription", "Failed to register Microsoft.KubernetesConfiguration")
output = utils.run("az extension add --name k8s-extension", "az k8s-extension installed", "Failed to install az k8s-extension")
output = utils.run("az extension update --name k8s-extension", "az k8s-extension updated", "Failed to update az k8s-extension")
output = utils.run("az extension add --name aks-preview", "az aks-preview extension installed", "Failed to install az aks-preview extension")
output = utils.run("az extension update --name aks-preview", "az aks-preview extension updated", "Failed to update az aks-preview extension")


<a id='2'></a>
### 2️⃣ Creates AKS and OpenAI resources, and deploys the application using Bicep 🦾
All resources deployed in this lab will be created within the designated resource group.   
The following step creates the AKS cluster and Azure OpenAI using a BICEP deployment.
It also deploys the application using the [Bicep Kubernetes extension (preview)](https://learn.microsoft.com/en-us/azure/aks/learn/quick-kubernetes-deploy-bicep-kubernetes-extension?tabs=azure-cli)

In [ ]:
if create_resources:
    utils.create_resource_group(create_resources, resource_group_name, resource_group_location)
    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "clusterName": { "value": aks_resource_name },
            "nodePoolCount": { "value": aks_nodepool_count },
            "nodePoolVMSize": { "value": aks_nodepool_vm_size },
            "openAIResourcePrefix": { "value": openai_resource_prefix },
            "openAIDeploymentName": { "value": openai_deployment_name },
            "openAIModelName": { "value": openai_model_name },
            "openAIModelVersion": { "value": openai_model_version }
        }
    }    
    with open('params.json', 'w') as bicep_parameters_file:
        bicep_parameters_file.write(json.dumps(bicep_parameters))

    output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json", 
                 f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")


<a id='3'></a>
### 3️⃣ Retrieve deployment outputs

In [ ]:
if create_resources:
    # retrieve deployment outputs
    output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")
    if output.success and output.json_data:
        aks_control_plane_fqdn = utils.get_deployment_output(output, 'controlPlaneFQDN', 'Control Plane FQDN')


<a id='4'></a>
### 4️⃣ Connect to the AKS cluster
Configure kubectl to connect to your Kubernetes cluster using the [az aks get-credentials](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-get-credentials) command. This command downloads credentials and configures the Kubernetes CLI to use them.

In [ ]:
output = utils.run(f"az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing",
             f"Credentials for AKS cluster '{aks_resource_name}' configured",
             f"Failed to configure credentials for AKS cluster '{aks_resource_name}'")


<a id='5'></a>
### 5️⃣ Retrieve the ingress IP for the application

Get the IP of the store admin web application and store front web application using the kubectl get service command.

In [ ]:
wait_time = 60 # in seconds
sleep_time = 5 # in seconds
request_timeout = 60 # in seconds
message = "Application is not ready yet. Check the logs."
while wait_time - sleep_time > 0:
    output = utils.run("kubectl get pods -l app=store-front -o jsonpath={..status.phase}")
    if output.success and output.text == "Running":
        output = utils.run("kubectl get service store-admin --output jsonpath={..status.loadBalancer.ingress[0]}")
        if output.success and output.json_data:
            ingress_ip = output.json_data['ip']
            utils.print_info(f"Service External IP: {ingress_ip}")
            try:
                response = requests.get(f"http://{ingress_ip}", timeout=request_timeout)
                if (response.status_code == 200):
                    utils.print_ok(f"Application ready @ http://{ingress_ip}")
                    message = None
                    break
            except:
                message = "The application is currently unresponsive..."
    wait_time -= sleep_time
    time.sleep(sleep_time)
if message:
    utils.print_warning(message)

<a id='6'></a>
### 6️⃣ Test the application by making a request to the AI service
You can also test the application by opening the user interface in a browser and creating a new product. OpenAI will assist in generating the product description.

In [ ]:
json = {"name":"octocat","tags":["octopus cat"]}

response = requests.post(f"http://{ingress_ip}/ai/generate/description", json = json)
utils.print_response(response)

<a id='clean'></a>
### 🗑️ Clean up resources
When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered. Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.